
# Rubin/LSST – NMSI Testability Notebook (DP1-ready)

This notebook provides **templates to run inside the Rubin Science Platform (RSP)** for testing falsifiable predictions of the NMSI framework using public Rubin data (DP1/operations) and Gaia cross-matches.

Modules included:
- **Test A** – Galactic vertical-wave harmonics & age-linked phase (Gaia×Rubin)
- **Test B** – Phase synchronization in star-forming regions (Kuramoto order)
- **Test C** – ISO non-gravitational acceleration phase signatures (Rubin Solar System Processing)


## 0) Setup & Imports

In [ ]:

import numpy as np, pandas as pd, matplotlib.pyplot as plt
from astropy.table import Table
from astropy.time import Time
from astropy import units as u
from astropy.coordinates import SkyCoord, Galactocentric
import pyvo
from astropy.timeseries import LombScargle
from scipy.signal import welch
print('Libraries loaded.')

## 1) Endpoints & basic config

In [ ]:

RUBIN_TAP = "https://data.lsst.cloud/api/tap"
GAIA_TAP  = "https://gea.esac.esa.int/tap-server/tap"
tap_rubin = pyvo.dal.TAPService(RUBIN_TAP)
tap_gaia  = pyvo.dal.TAPService(GAIA_TAP)
RA0, DEC0, RADIUS_DEG = 90.0, 0.0, 5.0
print('Endpoints set.')


## Test A — Vertical-wave **harmonics** & **age-linked phase** (Gaia × Rubin)

**Hypothesis (NMSI):** the MW disc vertical pattern is a **multi-harmonic oscillation**; young stars inherit the gas-phase (age-linked phase imprint).
**Falsifies NMSI if:** only a single tidal mode with no harmonic structure and no age-stratified phase lag.


In [ ]:

adql_gaia = f"""
SELECT TOP 50000
    g.source_id, g.ra, g.dec, g.pmra, g.pmdec, g.parallax, g.phot_g_mean_mag,
    vc.best_class_name AS var_type
FROM gaiadr3.gaia_source AS g
JOIN gaiadr3.vari_classifier_result AS vc
    ON g.source_id = vc.source_id
WHERE 1=INTERSECTS(
    CIRCLE('ICRS', {RA0}, {DEC0}, {RADIUS_DEG}),
    POINT('ICRS', g.ra, g.dec))
AND vc.best_class_name IN ('CEP', 'ACEP', 'T2CEP')
"""
gaia_job = tap_gaia.submit_job(adql_gaia); gaia_job.run()
gaia_df = gaia_job.fetch_result().to_table().to_pandas()
print(gaia_df.head(), len(gaia_df))

In [ ]:

adql_rubin = f"""
SELECT
    o.objectId, o.ra, o.dec, o.psfMag_g, o.psfMag_r
FROM dp01_dc2_catalogs.object AS o
WHERE 1=INTERSECTS(
    CIRCLE('ICRS', {RA0}, {DEC0}, {RADIUS_DEG}),
    POINT('ICRS', o.ra, o.dec))
"""
rubin_job = tap_rubin.submit_job(adql_rubin); rubin_job.run()
rubin_df = rubin_job.fetch_result().to_table().to_pandas()

from astropy.coordinates import match_coordinates_sky
gcoords = SkyCoord(gaia_df['ra'].values*u.deg, gaia_df['dec'].values*u.deg)
rcoords = SkyCoord(rubin_df['ra'].values*u.deg, rubin_df['dec'].values*u.deg)
idx, d2d, _ = gcoords.match_to_catalog_sky(rcoords)
sel = d2d < 0.5*u.arcsec
cross = pd.concat([gaia_df[sel].reset_index(drop=True),
                   rubin_df.iloc[idx[sel]].add_prefix('rb_').reset_index(drop=True)],
                  axis=1)
print(cross.head(), len(cross))

In [ ]:

# Z, W proxies and harmonic content (demo)
from astropy.coordinates import Galactocentric
dist = (1.0 / (cross['parallax'].values/1000.0)) * u.pc
coords = SkyCoord(ra=cross['ra'].values*u.deg, dec=cross['dec'].values*u.deg, distance=dist)
gc = coords.transform_to(Galactocentric())
Z = gc.z.to(u.kpc).value
pmra, pmdec = cross['pmra'].values * u.mas/u.yr, cross['pmdec'].values * u.mas/u.yr
vtan = 4.74 * (pmra.value**2 + pmdec.value**2)**0.5 * (dist.to(u.kpc).value)
W = vtan * np.sin(np.deg2rad(30.0))  # proxy

R = np.clip(dist.to(u.kpc).value, 0, 25)
bins = np.linspace(5, 20, 16); curves = []
for i in range(len(bins)-1):
    m = (R>=bins[i]) & (R<bins[i+1])
    if m.sum()<64: continue
    z_sorted = np.sort(Z[m]); w_sorted = W[m][np.argsort(Z[m])]
    f, Pxx = welch(w_sorted - np.nanmean(w_sorted), nperseg=min(256, m.sum()))
    curves.append((f,Pxx))

plt.figure(figsize=(6,4))
for f,P in curves: plt.loglog(f,P,alpha=0.3)
plt.xlabel('Spatial frequency'); plt.ylabel('Power'); plt.title('Vertical-wave harmonics'); plt.show()


## Test B — Phase synchronization in star-forming regions (Kuramoto order)

**Hypothesis (NMSI):** young-variable populations in SFRs show **phase-locking** (elevated Kuramoto order) vs. matched field stars.
**Falsifies NMSI if:** no excess coherence after controlling for cadence/systematics.


In [ ]:

adql_dia = f"""
SELECT TOP 50000
    d.objectId, d.band, d.midPointMjdTai AS mjd, d.psfFlux, d.psfFluxErr,
    o.ra, o.dec
FROM dp01_dc2_catalogs.forced_photometry AS d
JOIN dp01_dc2_catalogs.object AS o ON d.objectId = o.objectId
WHERE 1=INTERSECTS(
    CIRCLE('ICRS', {RA0}, {DEC0}, {RADIUS_DEG}),
    POINT('ICRS', o.ra, o.dec))
"""
dia_job = tap_rubin.submit_job(adql_dia); dia_job.run()
dia_df = dia_job.fetch_result().to_table().to_pandas()
print(dia_df.head())

In [ ]:

from astropy.timeseries import LombScargle
def estimate_phase(mjd, flux):
    t = np.array(mjd) - np.min(mjd)
    freq, power = LombScargle(t, flux).autopower(minimum_frequency=1/200.0, maximum_frequency=1/0.5)
    f0 = freq[np.argmax(power)]
    phase = (t * f0 * 2*np.pi) % (2*np.pi)
    return phase, 1/f0

def kuramoto_R(phases):
    return np.abs(np.mean(np.exp(1j*phases)))

R_vals = []
for oid, g in dia_df.groupby('objectId'):
    if len(g)<20: continue
    phase, period = estimate_phase(g['mjd'], g['psfFlux'])
    R_vals.append(kuramoto_R(phase))

R_vals = np.array(R_vals)
print('Kuramoto R median / 90th pct:', np.nanmedian(R_vals), np.nanpercentile(R_vals,90))
plt.hist(R_vals, bins=40); plt.xlabel('R'); plt.ylabel('N'); plt.title('Phase coherence in SFR field'); plt.show()


## Test C — ISO non-gravitational acceleration phase signatures

**Hypothesis (NMSI):** some interstellar objects (ISOs) show **quasi-periodic residuals** in orbit fits beyond standard outgassing.
**Falsifies NMSI if:** residuals are fully explained by standard non-grav models with no phase structure across the sample.


In [ ]:

adql_iso = """
SELECT TOP 20000
    sso.ssObjectId, sso.ra, sso.dec, sso.refEpoch, sso.e, sso.a, sso.i, sso.omega, sso.Omega, sso.M
FROM dp01_dc2_catalogs.ss_object AS sso
WHERE sso.e > 1.05
"""
iso_job = tap_rubin.submit_job(adql_iso); iso_job.run()
iso_df = iso_job.fetch_result().to_table().to_pandas()
print(iso_df.head())

In [ ]:

# Toy residual periodicity metric demo
rng = np.random.default_rng(42)
t = np.linspace(0, 200, 300)
resid = 0.002*np.sin(2*np.pi*t/30.0 + 1.2) + 0.001*rng.normal(size=t.size)

freq, power = LombScargle(t - t.min(), resid).autopower()
peak = power.max()
print('Example phase-structure metric (higher=more periodic):', float(peak))
plt.plot(t, resid, '.-'); plt.xlabel('time'); plt.ylabel('residual'); plt.title('ISO residuals (toy)'); plt.show()

print("Apply to real ISO ephemeris residuals post-fit. Significant periodic power across a subset supports NMSI; absence weakens it.")